In [2]:
import numpy as np
import random
import torch

def random_intensity_scaling(data, scale_range=(0.8, 1.2)):
    """
    Randomly scale the intensity of 3D data.

    Parameters:
        data (numpy.ndarray): The 3D data array to augment.
        scale_range (tuple): The range of scaling factors (min, max).

    Returns:
        numpy.ndarray: The augmented 3D data with adjusted intensities.
    """
    scale_factor = np.random.uniform(*scale_range)
    return data * scale_factor

def random_intensity_shift(data, shift_range=(-0.1, 0.1)):
    """
    Randomly shift the intensity of 3D data.

    Parameters:
        data (numpy.ndarray): The 3D data array to augment.
        shift_range (tuple): The range of intensity shifts (min, max).

    Returns:
        numpy.ndarray: The augmented 3D data with adjusted intensities.
    """
    shift = np.random.uniform(*shift_range)
    return data + shift

class RandomIntensityTransform:
    """
    Custom transform to adjust intensity for 16-bit 3D images with a probability.
    """
    def __init__(self, scale_range=(0.8, 1.2), shift_range=(-1000, 1000), clip_range=(0, 65535), p=0.5):
        """
        Parameters:
            scale_range (tuple): Range for intensity scaling factors.
            shift_range (tuple): Range for intensity shifts.
            clip_range (tuple): Min and max values for clipping (e.g., 0-65535 for 16-bit).
            p (float): Probability of applying the transformation.
        """
        self.scale_range = scale_range
        self.shift_range = shift_range
        self.clip_range = clip_range
        self.p = p

    def __call__(self, tensor):
        if random.random() < self.p:
            scale_factor = random.uniform(*self.scale_range)
            tensor = tensor * scale_factor
            
            shift = random.uniform(*self.shift_range)
            tensor = tensor + shift
            
            tensor = torch.clamp(torch.from_numpy(tensor), min=self.clip_range[0], max=self.clip_range[1])
        return tensor


In [3]:
shift_range = 20
transform = RandomIntensityTransform(
    scale_range=(0.3, 2.5),
    shift_range=(-shift_range, shift_range),
    p=1.0
)

In [4]:
from skimage import io
import torchio as tio
test_block_path = "/scratch/dataset_patch_128_steps_64_clip_int/train/images/698928_image_block_154.tif"
block = io.imread(test_block_path)
print(block.shape, block.min(), block.max(), np.mean(block))

(128, 128, 128) 0 232 3.734201431274414


In [6]:
subject = tio.Subject(
    image=tio.ScalarImage(tensor=np.expand_dims(block, axis=0))
)

In [30]:
transform = tio.RandomGamma(log_gamma=(-0.2, 0.5), p=1.0)
transformed = transform(subject).image.data.numpy()
print(transformed.shape, type(transformed))
np.save("/results/gamma_test.npy", transformed)

(1, 128, 128, 128) <class 'numpy.ndarray'>


In [24]:
new_block = transform(block)
new_block = new_block.cpu().numpy()
print(new_block.shape, new_block.dtype, type(new_block))
print(new_block.shape, new_block.min(), new_block.max(), np.mean(new_block))

np.save("/results/check_intensity.npy", new_block)

(128, 128, 128) float64 <class 'numpy.ndarray'>
(128, 128, 128) 6.859482220037663 358.24114665620954 12.515214609094297
